In [22]:
import numpy
import matplotlib.pyplot
%matplotlib inline
# contains sigmoid function
import scipy.special



In [30]:
class NeuralNetwork:
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate) -> None:
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes

        #learning rate
        self.lr = learningrate

        # link weight matrices wih (weights in to hidden layer) and who (weights hidden to output layer)
        # weights inside the arrays are w_i_j where the link is from the node i to the node j in the next layer
        # w11 w21
        # w12 w22 etc

        # self.wih = (numpy.random.rand(self.hnodes, self.inodes) - 0.5)
        # self.who = (numpy.random.rand(self.onodes, self.hnodes) - 0.5)
        self.wih = numpy.random.normal( 0.0, pow(self.inodes, -0.5), (self.hnodes , self.inodes))
        self.who = numpy.random.normal( 0.0, pow(self.hnodes, -0.5), (self.onodes , self.hnodes))

        self.activation_function = lambda x: scipy.special.expit(x)


        pass

    def train(self, inputs_list, targets_list):
        #convert inputs list into 2D array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T

        #calculate signal into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        #calculate the signals emerging from the hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)

        #calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        #calculate the signals emerging from the final output layer
        final_outputs = self.activation_function(final_inputs)

        #calculate error (target - actual)
        output_errors = targets - final_outputs

        # backpropagate errors to hidden layer
        hidden_errors = numpy.dot(self.who.T, output_errors)

        #to refine the weights between the output & hidden layer, we use "output_errors"
        self.who += self.lr * numpy.dot( (output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        #to refine the weights between the hidden & the input layer, we use "hidden_errors"
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))



        pass

    def query(self, inputs_list):

        #convert input list into 2D array
        inputs = numpy.array(inputs_list, ndmin=2).T

        hidden_inputs = numpy.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)

        final_inputs = numpy.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        return final_outputs


In [35]:
input_nodes = 3
hidden_nodes = 3
output_nodes = 3

learning_rate = 0.3

n = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [38]:
n.query([1.0, 0.5, -1.5])

array([[0.54999461],
       [0.48024805],
       [0.59944428]])